In [ ]:
mod utils;

fn main() {
    let notebooks = [
        "03_stack.ipynb",
        "04_memory.ipynb",
        "05_storage.ipynb",
        "07_opcodes.ipynb",
        "07a_opcodes/01_stop.ipynb",
        "07a_opcodes/02_math.ipynb",
        "07a_opcodes/03_comparisions.ipynb",
        "07a_opcodes/04_logic.ipynb",
        "07a_opcodes/05_bit.ipynb",
        "07a_opcodes/06_misc.ipynb",
        "07a_opcodes/07_environment.ipynb",
        "07a_opcodes/08_pop.ipynb",
        "07a_opcodes/09_memory.ipynb",
        "07a_opcodes/10_storage.ipynb",
        "07a_opcodes/11_jump.ipynb",
        "07a_opcodes/12_push.ipynb",
        "07a_opcodes/13_dup.ipynb",
        "07a_opcodes/14_swap.ipynb",
        "07a_opcodes/15_log.ipynb",
        "07a_opcodes/16_contract.ipynb",
        "07a_opcodes/17_transient.ipynb",
    ];

    utils::import_notebooks(&notebooks); // Call function from utils module
}


# EVM

Now we are going to put everything together.

### Program Counter

The program counter indexes into the bytecode. It tells the EVM which opcode to execute next

In [ ]:
pub struct EVM {
    pub pc: usize,
    pub stack: Stack,
    pub memory: Memory,
    pub storage: Storage,

    pub program: Vec<u8>,
    pub gas: u64,
    pub value: u64,
    pub calldata: Vec<u8>,

    pub stop_flag: bool,
    pub revert_flag: bool,

    pub returndata: Vec<u8>,
    pub logs: Vec<Log>,
}

impl EVM {
    pub fn new(program: Vec<u8>, gas: u64, value: u64, calldata: Vec<u8>) -> Self {
        Self {
            pc: 0,
            stack: Stack::new(),
            memory: Memory::new(),
            storage: Storage::new(),

            program,
            gas,
            value,
            calldata,

            stop_flag: false,
            revert_flag: false,

            returndata: vec![],
            logs: vec![],
        }
    }

    pub fn peek(&self) -> Option<u8> {
        self.program.get(self.pc).copied()
    }

    pub fn gas_dec(&mut self, amount: u64) -> Result<(), &'static str> {
        if self.gas < amount {
            return Err("out of gas");
        }
        self.gas -= amount;
        Ok(())
    }

    pub fn should_execute_next_opcode(&self) -> bool {
        self.pc < self.program.len() && !self.stop_flag && !self.revert_flag
    }

    pub fn run(&mut self) {
    while self.should_execute_next_opcode() {
        let op = self.program[self.pc];

        match op {
            // Stop & Control Flow
            STOP => stop(self),

            // Math Operations
            ADD => add(self),
            MUL => mul(self),
            SUB => sub(self),
            DIV => div(self),
            SDIV => sdiv(self),
            MOD => modu(self),
            SMOD => smod(self),
            ADDMOD => addmod(self),
            MULMOD => mulmod(self),
            EXP => exp(self),
            SIGNEXTEND => signextend(self),

            // Comparison Operations
            LT => lt(self),
            GT => gt(self),
            SLT => slt(self),
            SGT => sgt(self),
            EQ => eq(self),
            ISZERO => iszero(self),

            // Logic Operations
            AND => _and(self),
            OR => _or(self),
            XOR => _xor(self),
            NOT => _not(self),

            // Bit Operations
            BYTE => byte(self),
            SHL => shl(self),
            SHR => shr(self),
            SAR => sar(self),

            // SHA3
            SHA3 => sha3(self),

            // Environment Information
            ADDRESS => address(self),
            BALANCE => balance(self),
            ORIGIN => origin(self),
            CALLER => caller(self),
            CALLVALUE => callvalue(self),
            CALLDATALOAD => calldataload(self),
            CALLDATASIZE => calldatasize(self),
            CALLDATACOPY => calldatacopy(self),
            CODESIZE => codesize(self),
            CODECOPY => codecopy(self),
            GASPRICE => gasprice(self),
            EXTCODESIZE => extcodesize(self),
            EXTCODECOPY => extcodecopy(self),
            RETURNDATASIZE => returndatasize(self),
            RETURNDATACOPY => returndatacopy(self),
            EXTCODEHASH => extcodehash(self),
            BLOCKHASH => blockhash(self),
            COINBASE => coinbase(self),
            TIMESTAMP => timestamp(self),
            NUMBER => number(self),
            DIFFICULTY => difficulty(self),
            GASLIMIT => gaslimit(self),
            CHAINID => chainid(self),
            SELFBALANCE => selfbalance(self),
            BASEFEE => basefee(self),

            // Stack Operations
            POP => _pop(self),

            // Memory Operations
            MLOAD => mload(self),
            MSTORE => mstore(self),
            MSTORE8 => mstore8(self),

            // Storage Operations
            SLOAD => sload(self),
            SSTORE => sstore(self),

            // Jump Operations
            JUMP => jump(self),
            JUMPI => jumpi(self),
            PC => pc(self),
            JUMPDEST => jumpdest(self),

            // Transient Storage Operations
            TLOAD => tload(self),
            TSTORE => tstore(self),

            // Push Operations (0x60-0x7F)
            PUSH1 => _push(self, 1),   PUSH2 => _push(self, 2),
            PUSH3 => _push(self, 3),   PUSH4 => _push(self, 4),
            PUSH5 => _push(self, 5),   PUSH6 => _push(self, 6),
            PUSH7 => _push(self, 7),   PUSH8 => _push(self, 8),
            PUSH9 => _push(self, 9),   PUSH10 => _push(self, 10),
            PUSH11 => _push(self, 11), PUSH12 => _push(self, 12),
            PUSH13 => _push(self, 13), PUSH14 => _push(self, 14),
            PUSH15 => _push(self, 15), PUSH16 => _push(self, 16),
            PUSH17 => _push(self, 17), PUSH18 => _push(self, 18),
            PUSH19 => _push(self, 19), PUSH20 => _push(self, 20),
            PUSH21 => _push(self, 21), PUSH22 => _push(self, 22),
            PUSH23 => _push(self, 23), PUSH24 => _push(self, 24),
            PUSH25 => _push(self, 25), PUSH26 => _push(self, 26),
            PUSH27 => _push(self, 27), PUSH28 => _push(self, 28),
            PUSH29 => _push(self, 29), PUSH30 => _push(self, 30),
            PUSH31 => _push(self, 31), PUSH32 => _push(self, 32),

            // Dup Operations (0x80-0x8F)
            DUP1 => _dup(self, 1),   DUP2 => _dup(self, 2),
            DUP3 => _dup(self, 3),   DUP4 => _dup(self, 4),
            DUP5 => _dup(self, 5),   DUP6 => _dup(self, 6),
            DUP7 => _dup(self, 7),   DUP8 => _dup(self, 8),
            DUP9 => _dup(self, 9),   DUP10 => _dup(self, 10),
            DUP11 => _dup(self, 11), DUP12 => _dup(self, 12),
            DUP13 => _dup(self, 13), DUP14 => _dup(self, 14),
            DUP15 => _dup(self, 15), DUP16 => _dup(self, 16),

            // Swap Operations (0x90-0x9F)
            SWAP1 => _swap(self, 1),   SWAP2 => _swap(self, 2),
            SWAP3 => _swap(self, 3),   SWAP4 => _swap(self, 4),
            SWAP5 => _swap(self, 5),   SWAP6 => _swap(self, 6),
            SWAP7 => _swap(self, 7),   SWAP8 => _swap(self, 8),
            SWAP9 => _swap(self, 9),   SWAP10 => _swap(self, 10),
            SWAP11 => _swap(self, 11), SWAP12 => _swap(self, 12),
            SWAP13 => _swap(self, 13), SWAP14 => _swap(self, 14),
            SWAP15 => _swap(self, 15), SWAP16 => _swap(self, 16),

            // Log Operations
            LOG0 => log0(self),
            LOG1 => log1(self),
            LOG2 => log2(self),
            LOG3 => log3(self),
            LOG4 => log4(self),

            // Contract Operations
            CREATE => create(self),
            CALL => call(self),
            CALLCODE => callcode(self),
            RETURN => _return(self),
            DELEGATECALL => delegatecall(self),
            CREATE2 => create2(self),
            STATICCALL => staticcall(self),
            REVERT => revert(self),
            INVALID => invalid(self),
            SELFDESTRUCT => selfdestruct(self),

            _ => panic!("Unknown opcode: 0x{:x}", op),
        }
    }
}

    pub fn reset(&mut self) {
        self.pc = 0;
        self.stack = Stack::new();
        self.memory = Memory::new();
        self.storage = Storage::new();
    }
}
